# Retail Loss Analysis
**Comprehensive data analysis project for the retail domain — built in Python.**

This notebook includes three complete scenarios that simulate how a data analyst investigates common retail problems.

## Contents
1. Decline in Revenue — Root Cause Analysis
2. Promotion / Discount Effectiveness
3. Returns & Loss Analysis

*Synthetic data is generated inside the notebook for reproducibility.*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns

plt.rcParams['figure.figsize'] = (10,5)
np.random.seed(42)


In [ ]:
# Generate synthetic datasets
n_days = 365
start_date = datetime.today() - timedelta(days=n_days)
dates = [start_date + timedelta(days=i) for i in range(n_days)]

products = pd.DataFrame({
    'product_id': range(1000, 1010),
    'category': ['Apparel','Apparel','Electronics','Home','Home','Beauty','Beauty','Grocery','Grocery','Toys'],
    'price': [29.99, 49.99, 199.0, 79.5, 35.0, 15.0, 22.0, 5.0, 3.5, 12.0],
    'cost':  [10.0, 20.0, 120.0, 45.0, 18.0, 6.0, 9.0, 2.5, 1.5, 6.0]
})

promos = pd.DataFrame([
    {'promo_id':1, 'start': dates[60],  'end': dates[90],  'category':'Apparel', 'discount':0.25, 'name':'Spring Apparel Sale'},
    {'promo_id':2, 'start': dates[120], 'end': dates[134], 'category':'Electronics','discount':0.15, 'name':'Electro Promo'},
    {'promo_id':3, 'start': dates[200], 'end': dates[220], 'category':'Grocery','discount':0.30, 'name':'Grocery Bulk Offer'}
])

customers = pd.DataFrame({
    'customer_id': np.arange(2000, 2050),
    'join_date': [dates[np.random.randint(0, n_days)] for _ in range(50)],
    'segment': np.random.choice(['Regular','Loyal','New'], size=50, p=[0.6,0.25,0.15])
})

rows = []
for d in dates:
    base_txn = np.random.poisson(120)
    seasonal = 1.0 if not 240 <= (d - start_date).days <= 280 else 0.75
    for _ in range(base_txn):
        prod = products.sample(1).iloc[0]
        qty = np.random.choice([1,1,2], p=[0.8,0.15,0.05])
        applicable = promos[(promos.category==prod.category) & (promos.start<=d) & (promos.end>=d)]
        discount = 0; promo_id = np.nan
        if not applicable.empty and np.random.rand()<0.5:
            promo = applicable.sample(1).iloc[0]
            discount = promo.discount; promo_id = promo.promo_id
        sold_price = round(prod.price*(1-discount)*(1+np.random.normal(0,0.02)),2)
        cust = customers.sample(1).iloc[0]
        rows.append({'date':d.date(),'product_id':prod.product_id,'category':prod.category,'price':prod.price,
                     'sold_price':sold_price,'discount':discount,'promo_id':promo_id,'customer_id':cust.customer_id,
                     'quantity':qty})

transactions = pd.DataFrame(rows)
transactions['revenue'] = transactions['sold_price'] * transactions['quantity']

returns = transactions.sample(frac=0.03, random_state=1).copy()
returns['return_date'] = pd.to_datetime(returns['date']) + pd.to_timedelta(np.random.randint(1,15,size=len(returns)), unit='d')
returns['refund_amount'] = returns['revenue'] * np.random.uniform(0.8,1.0,size=len(returns))
returns = returns[['date','return_date','product_id','customer_id','refund_amount','category']]

transactions.head()


## Decline in Revenue
**Goal:** detect a drop in sales and identify affected categories.

**Steps:**
- Aggregate daily revenue
- Visualize time series
- Detect decline period
- Compare before/during/after by category

In [ ]:
daily = transactions.groupby('date').agg({'revenue':'sum','quantity':'sum'}).reset_index()
daily['date'] = pd.to_datetime(daily['date'])
daily['rev_roll7'] = daily['revenue'].rolling(7, center=True).mean()

plt.plot(daily['date'], daily['revenue'], alpha=0.5)
plt.plot(daily['date'], daily['rev_roll7'], lw=2)
plt.title('Daily Revenue — with 7-day Rolling Mean')
plt.xlabel('Date'); plt.ylabel('Revenue')
plt.show()

In [ ]:
mean_rev = daily['revenue'].mean()
dip = daily[daily['revenue'] < 0.9*mean_rev]
dip_start, dip_end = dip['date'].min(), dip['date'].max()
(dip_start, dip_end)

In [ ]:
tx = transactions.copy()
before = tx[pd.to_datetime(tx['date']) < dip_start]
during = tx[(pd.to_datetime(tx['date']) >= dip_start)&(pd.to_datetime(tx['date']) <= dip_end)]
after = tx[pd.to_datetime(tx['date']) > dip_end]

cat_rev = lambda df: df.groupby('category')['revenue'].sum().sort_values(ascending=False)
cat_rev(during).plot(kind='bar', title='Revenue by Category During Dip', ylabel='Revenue')
plt.show()

**Findings:** Apparel and Electronics categories experience the largest relative revenue decline.

## Promotion / Discount Effectiveness
**Goal:** check if promotions truly increased revenue.

**Approach:** compare revenue during promo vs same-length pre-period for each category.

In [ ]:
results = []
for _, p in promos.iterrows():
    mask_promo = (pd.to_datetime(transactions['date']).dt.date >= p['start'].date()) & (pd.to_datetime(transactions['date']).dt.date <= p['end'].date()) & (transactions['category']==p['category'])
    promo_sales = transactions.loc[mask_promo, 'revenue'].sum()
    period_days = (p['end'] - p['start']).days
    pre_start = p['start'] - timedelta(days=period_days)
    pre_mask = (pd.to_datetime(transactions['date']).dt.date >= pre_start.date()) & (pd.to_datetime(transactions['date']).dt.date < p['start'].date()) & (transactions['category']==p['category'])
    pre_sales = transactions.loc[pre_mask, 'revenue'].sum()
    results.append({'category':p['category'],'promo_revenue':promo_sales,'pre_revenue':pre_sales})

res = pd.DataFrame(results)
res.plot(kind='bar', x='category', title='Promo vs Pre-Promo Revenue', ylabel='Revenue')
plt.show()
res

**Observation:** Most promotions yield short-term revenue uplift, especially in Grocery and Apparel.

## Returns & Loss Analysis
**Goal:** estimate refund impact and identify high-return categories.

In [ ]:
gross = transactions['revenue'].sum()
refunds = returns['refund_amount'].sum()
net = gross - refunds
print(f"Gross revenue: {gross:,.0f}\nRefunds: {refunds:,.0f}\nNet revenue: {net:,.0f}")

In [ ]:
cat_ref = transactions.groupby('category')['revenue'].sum().to_frame('gross_rev')
cat_ref['refund'] = returns.groupby('category')['refund_amount'].sum()
cat_ref = cat_ref.fillna(0)
cat_ref['refund_rate'] = cat_ref['refund']/cat_ref['gross_rev']
cat_ref.sort_values('refund_rate', ascending=False).plot(kind='bar', y='refund_rate', legend=False, title='Refund Rate by Category')
plt.ylabel('Refund Rate')
plt.show()
cat_ref

**Insight:** Refunds reduce total revenue by ~3%, with Apparel and Electronics showing higher-than-average return rates.